<a href="https://colab.research.google.com/github/morozovsolncev/gravitation/blob/main/%D0%93%D1%80%D0%B0%D0%B2%D0%B8%D1%82%D0%B0%D1%86%D0%B8%D1%8F_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#Иерархическая модель гравитации на основе теории БПП (Беспредельного Поля Потенций)
#Версия 2.2

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from matplotlib.animation import FuncAnimation
import matplotlib
matplotlib.use('Agg')

class HierarchicalBPPSimulator:
    def __init__(self):
        """Инициализация симулятора с иерархической структурой системы"""
        # Физические константы (SI)
        self.G = 6.67430e-11      # Гравитационная постоянная
        self.c = 299792458         # Скорость света

        # Массы тел (кг)
        sun_mass = 1.989e30        # Масса Солнца
        earth_mass = 5.972e24      # Масса Земли
        moon_mass = 7.348e22       # Масса Луны

        # Характерные расстояния (м)
        earth_sun_dist = 1.496e11  # Большая полуось орбиты Земли
        moon_earth_dist = 3.844e8  # Большая полуось орбиты Луны

        # Расчет барицентра системы Земля-Луна
        total_mass = earth_mass + moon_mass
        barycenter_offset = moon_mass * moon_earth_dist / total_mass

        # Орбитальные скорости (м/с)
        earth_sun_velocity = np.sqrt(self.G * sun_mass / earth_sun_dist)  # Орбитальная скорость Земли
        orbital_velocity = np.sqrt(self.G * total_mass / moon_earth_dist)  # Орбитальная скорость Луны

        # Иерархическая структура системы
        self.system = {
            "Sun": {
                "name": "Sun",
                "mass": sun_mass,
                "position": np.array([0.0, 0.0]),
                "velocity": np.array([0.0, 0.0]),
                "children": ["Earth"],
                "local_time_factor": 1.0,  # Фактор локального времени
                "level": 0                  # Иерархический уровень (корневой)
            },
            "Earth": {
                "name": "Earth",
                "mass": earth_mass,
                "position": np.array([earth_sun_dist - barycenter_offset, 0.0]),
                "velocity": np.array([0.0, earth_sun_velocity - orbital_velocity * moon_mass / total_mass]),
                "parent": "Sun",
                "children": ["Moon"],
                "local_time_factor": 1.0,    # Фактор локального времени
                "level": 1                   # Иерархический уровень (планета)
            },
            "Moon": {
                "name": "Moon",
                "mass": moon_mass,
                "position": np.array([earth_sun_dist + moon_earth_dist - barycenter_offset, 0.0]),
                "velocity": np.array([0.0, earth_sun_velocity + orbital_velocity * earth_mass / total_mass]),
                "parent": "Earth",
                "local_time_factor": 1.000075,  # Учет релятивистского замедления времени
                "level": 2                     # Иерархический уровень (спутник)
            }
        }

        # Параметры симуляции
        self.t_max = 27.3 * 24 * 3600  # Время симуляции (1 лунный месяц)
        self.dt = 1800                  # Шаг интегрирования (30 минут)
        self.t_points = np.arange(0, self.t_max, self.dt)

        # Калибровочный коэффициент для гравитации
        self.grav_coeff = 0.5  # Эмпирический параметр для соответствия наблюдаемым орбитам

        # Диагностические данные
        self.acceleration_data = {name: [] for name in self.system}

    def get_resonance_strength(self, target, source, r, v_rel):
        """Вычисление вероятности резонанса между двумя паттернами"""
        # Базовый гравитационный потенциал
        base_res = 2 * self.G * source["mass"] / (self.c**2 * r)

        # Иерархический коэффициент
        level_diff = abs(target["level"] - source["level"])
        hierarchy_boost = 10**(3 - min(level_diff, 3))

        # Релятивистский фактор (замедление времени)
        time_dilation = np.exp(-base_res * hierarchy_boost)

        # Допплеровский фактор (влияние относительной скорости)
        r_vec = source["position"] - target["position"]
        if r > 1e-10:  # Защита от деления на ноль
            vr = np.dot(v_rel, r_vec) / r
        else:
            vr = 0
        doppler_factor = 1 + vr / self.c

        # Итоговая вероятность резонанса
        p_res = np.exp(base_res * hierarchy_boost) * time_dilation * doppler_factor

        return p_res

    def acceleration(self, body_name):
        """Расчет ускорения тела в рамках теории БПП"""
        body = self.system[body_name]
        a_total = np.zeros(2)

        # Центробежное ускорение (для вращающихся систем)
        if "parent" in body:
            parent = self.system[body["parent"]]
            r_vec = body["position"] - parent["position"]
            r = np.linalg.norm(r_vec)
            if r > 1e-10:
                # Классическое центробежное ускорение
                v_tangent = np.array([-body["velocity"][1], body["velocity"][0]])
                centrifugal = np.dot(body["velocity"], v_tangent) / r**2 * r_vec
                a_total += centrifugal

        # Расчет вкладов от всех других тел системы
        for source_name, source in self.system.items():
            if source_name == body_name:
                continue

            r_vec = source["position"] - body["position"]
            r = np.linalg.norm(r_vec)
            if r < 1e6:  # Игнорируем очень близкие объекты
                continue

            v_rel = source["velocity"] - body["velocity"]
            direction = r_vec / r

            # Вероятность резонанса
            p_res = self.get_resonance_strength(body, source, r, v_rel)

            # Градиент вероятности резонанса
            dp_dr = p_res * (2 * self.G * source["mass"]) / (self.c**2 * r**2)

            # Релятивистская поправка (прецессия перигелия)
            rel_correction = 1 + 3 * self.G * source["mass"] / (self.c**2 * r)

            # Усиление релятивистских эффектов на малых расстояниях
            if r < 2e8:
                rel_correction *= 1.5

            # Ускорение согласно основному уравнению БПП
            a_mag = self.grav_coeff * self.c**2 * dp_dr / max(p_res, 1e-20) * rel_correction
            a_total += a_mag * direction

        # Сохраняем для диагностики
        self.acceleration_data[body_name].append(a_total.copy())

        return a_total

    def derivatives(self, t, y):
        """Функция для решения системы дифференциальных уравнений"""
        names = list(self.system.keys())
        n = len(names)
        dy_dt = np.zeros(4 * n)

        # Обновление позиций и скоростей из вектора состояния
        for i, name in enumerate(names):
            self.system[name]["position"] = y[2*i:2*i+2]
            self.system[name]["velocity"] = y[2*n + 2*i:2*n + 2*i+2]

        # Расчет производных
        for i, name in enumerate(names):
            time_factor = self.system[name].get("local_time_factor", 1.0)

            # Производная позиции = скорость * фактор времени
            dy_dt[2*i:2*i+2] = self.system[name]["velocity"] * time_factor

            # Производная скорости = ускорение * фактор времени
            a = self.acceleration(name)
            dy_dt[2*n + 2*i:2*n + 2*i+2] = a * time_factor

        return dy_dt

    def run_simulation(self):
        """Запуск основной симуляции движения тел"""
        names = list(self.system.keys())
        n = len(names)

        # Начальные условия
        y0 = np.zeros(4 * n)
        for i, name in enumerate(names):
            y0[2*i:2*i+2] = self.system[name]["position"]
            y0[2*n + 2*i:2*n + 2*i+2] = self.system[name]["velocity"]

        # Решение системы ОДУ
        try:
            solution = solve_ivp(
                self.derivatives,
                [0, self.t_max],
                y0,
                t_eval=self.t_points,
                method='DOP853',  # Высокоточный метод
                rtol=1e-9,
                atol=1e-9
            )

            # Обработка результатов
            trajectories = {}
            velocities = {}
            for i, name in enumerate(names):
                trajectories[name] = solution.y[2*i:2*i+2].T
                velocities[name] = solution.y[2*n + 2*i:2*n + 2*i+2].T

            # Относительная траектория Луны относительно Земли
            trajectories["Moon_relative"] = trajectories["Moon"] - trajectories["Earth"]
            velocities["Moon_relative"] = velocities["Moon"] - velocities["Earth"]

            return trajectories, velocities

        except Exception as e:
            print(f"Ошибка при решении ОДУ: {e}")
            # Возвращаем начальные позиции в случае ошибки
            trajectories = {}
            velocities = {}
            for name in names:
                trajectories[name] = np.tile(self.system[name]["position"], (len(self.t_points), 1))
                velocities[name] = np.tile(self.system[name]["velocity"], (len(self.t_points), 1))
            trajectories["Moon_relative"] = trajectories["Moon"] - trajectories["Earth"]
            velocities["Moon_relative"] = velocities["Moon"] - velocities["Earth"]
            return trajectories, velocities

    def create_animation(self, moon_rel):
        """Создание анимации движения Луны вокруг Земли"""
        fig, ax = plt.subplots(figsize=(10, 10))
        range_val = 1.5 * np.max(np.abs(moon_rel)) if len(moon_rel) > 0 else 1e9
        ax.set_xlim(-range_val, range_val)
        ax.set_ylim(-range_val, range_val)
        ax.set_title('Движение Луны вокруг Земли в БПП модели', fontsize=14)
        ax.set_xlabel('x, м', fontsize=12)
        ax.set_ylabel('y, м', fontsize=12)
        ax.grid(True, linestyle='--', alpha=0.5)

        # Земля
        earth_radius = range_val * 0.02
        earth = plt.Circle((0, 0), earth_radius, color='blue', alpha=0.7)
        ax.add_patch(earth)

        # Луна и орбита
        moon_dot, = ax.plot([], [], 'ro', ms=8)
        orbit_line, = ax.plot([], [], 'r-', alpha=0.3)
        time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=12,
                           bbox=dict(facecolor='white', alpha=0.7))

        # Инициализация
        def init():
            moon_dot.set_data([], [])
            orbit_line.set_data([], [])
            time_text.set_text('')
            return moon_dot, orbit_line, time_text

        # Обновление кадра
        def update(frame):
            # Траектория за последние 200 шагов
            start_idx = max(0, frame - 200)
            x = moon_rel[start_idx:frame+1, 0]
            y = moon_rel[start_idx:frame+1, 1]

            orbit_line.set_data(x, y)
            moon_dot.set_data([moon_rel[frame, 0]], [moon_rel[frame, 1]])

            # Время в днях
            days = frame * self.dt / (24 * 3600)
            time_text.set_text(f'День: {days:.2f}')

            return moon_dot, orbit_line, time_text

        # Создание анимации
        try:
            ani = FuncAnimation(
                fig, update, frames=len(moon_rel),
                init_func=init, blit=True, interval=30
            )

            # Сохранение
            ani.save('bpp_moon_animation.mp4', writer='ffmpeg', fps=30, dpi=100)
            print("Анимация сохранена как 'bpp_moon_animation.mp4'")
        except Exception as e:
            print(f"Ошибка при создании анимации: {e}")
        finally:
            plt.close(fig)

    def plot_results(self, trajectories, velocities):
        """Расширенная визуализация с параметрами БПП"""
        if "Moon_relative" not in trajectories:
            print("Нет данных для визуализации")
            return

        moon_rel = trajectories["Moon_relative"]
        earth_sun = trajectories["Earth"] - trajectories["Sun"]

        # 1. Основной график орбиты
        fig, ax = plt.subplots(figsize=(12, 10))

        # Идеальная круговая орбита для сравнения
        real_orbit_radius = 3.844e8
        circle = plt.Circle((0, 0), real_orbit_radius, fill=False,
                           color='gray', linestyle='--', alpha=0.7,
                           label='Идеальная орбита')
        ax.add_artist(circle)

        # Земля
        ax.plot(0, 0, 'bo', ms=15, label='Земля')

        # Орбита Луны с цветовой кодировкой резонанса
        resonance_strength = self.calculate_resonance(trajectories)
        scatter = ax.scatter(moon_rel[:, 0], moon_rel[:, 1], c=resonance_strength,
                            cmap='viridis', s=10, alpha=0.7)
        plt.colorbar(scatter, label='Сила резонанса')

        # Направление на Солнце
        sun_dirs = earth_sun / np.linalg.norm(earth_sun, axis=1)[:, np.newaxis]
        for i in range(0, len(sun_dirs), len(sun_dirs)//10):
            ax.arrow(0, 0, sun_dirs[i,0]*real_orbit_radius*0.7,
                     sun_dirs[i,1]*real_orbit_radius*0.7,
                     head_width=1e7, head_length=2e7, fc='orange', ec='orange')

        # Настройка графика
        ax.set_title('Орбита Луны с параметрами БПП', fontsize=16)
        ax.set_xlabel('x, м', fontsize=14)
        ax.set_ylabel('y, м', fontsize=14)
        ax.grid(True, linestyle='--', alpha=0.7)
        ax.legend(fontsize=12, loc='upper right')
        ax.axis('equal')
        max_range = 1.3 * np.max(np.abs(moon_rel))
        ax.set_xlim(-max_range, max_range)
        ax.set_ylim(-max_range, max_range)
        plt.savefig('bpp_moon_orbit_enhanced.png', dpi=200)
        plt.close()

        # 2. График параметров БПП
        self.plot_bpp_parameters(trajectories, velocities)

    def calculate_resonance(self, trajectories):
        """Расчет силы резонанса для Луны"""
        resonance = []
        moon_pos = trajectories["Moon"]
        earth_pos = trajectories["Earth"]
        sun_pos = trajectories["Sun"]

        for i in range(len(moon_pos)):
            # Расстояния
            r_earth = np.linalg.norm(moon_pos[i] - earth_pos[i])
            r_sun = np.linalg.norm(moon_pos[i] - sun_pos[i])

            # Вероятность резонанса с Землей
            p_earth = np.exp(2 * self.G * self.system["Earth"]["mass"] / (self.c**2 * r_earth))

            # Вероятность резонанса с Солнцем
            p_sun = np.exp(2 * self.G * self.system["Sun"]["mass"] / (self.c**2 * r_sun))

            # Иерархическая композиция
            total_res = p_earth * 10**3 + p_sun * 1  # κ_earth = 10^3, κ_sun = 1
            resonance.append(total_res)

        return np.array(resonance)

    def plot_bpp_parameters(self, trajectories, velocities):
        """Графики специфических параметров БПП"""
        time_days = self.t_points / (24 * 3600)

        # 1. Расстояние до Земли
        moon_rel = trajectories["Moon_relative"]
        earth_distance = np.linalg.norm(moon_rel, axis=1)

        # 2. Сила резонанса
        resonance_strength = self.calculate_resonance(trajectories)

        # 3. Энергия актуализации
        moon_vel = velocities["Moon_relative"]
        kinetic_energy = 0.5 * np.linalg.norm(moon_vel, axis=1)**2
        potential_energy = -self.G * self.system["Earth"]["mass"] / earth_distance
        total_energy = kinetic_energy + potential_energy

        # 4. Угол Солнце-Земля-Луна
        earth_sun = trajectories["Sun"] - trajectories["Earth"]
        sun_dirs = earth_sun / np.linalg.norm(earth_sun, axis=1)[:, np.newaxis]
        moon_dirs = moon_rel / np.linalg.norm(moon_rel, axis=1)[:, np.newaxis]
        sun_moon_angle = np.degrees(np.arccos(np.clip(
            np.sum(sun_dirs * moon_dirs, axis=1), -1.0, 1.0)))

        fig, axs = plt.subplots(4, 1, figsize=(14, 16), sharex=True)

        # График 1: Расстояние и резонанс
        axs[0].plot(time_days, earth_distance, 'b-', label='Расстояние до Земли')
        axs[0].axhline(y=3.844e8, color='r', linestyle='--', label='Реальное значение')
        axs[0].set_ylabel('Расстояние, м', fontsize=12)
        axs[0].legend(loc='upper right')
        ax_res = axs[0].twinx()
        ax_res.plot(time_days, resonance_strength, 'g-', alpha=0.7, label='Сила резонанса')
        ax_res.set_ylabel('Резонанс', color='g', fontsize=12)
        ax_res.legend(loc='upper left')

        # График 2: Энергии
        axs[1].plot(time_days, kinetic_energy, 'b-', label='Кинетическая')
        axs[1].plot(time_days, potential_energy, 'r-', label='Потенциальная')
        axs[1].plot(time_days, total_energy, 'g-', label='Полная')
        axs[1].set_ylabel('Энергия, Дж', fontsize=12)
        axs[1].legend()

        # График 3: Угол к Солнцу
        axs[2].plot(time_days, sun_moon_angle, 'm-')
        axs[2].set_ylabel('Угол Солнце-Луна, °', fontsize=12)
        axs[2].set_ylim(0, 180)

        # График 4: Эксцентриситет и большая полуось
        orbital = self.calculate_orbital_elements(trajectories, velocities)
        axs[3].plot(time_days, orbital['semi_major_axis'], 'b-', label='Большая полуось')
        axs[3].axhline(y=3.844e8, color='b', linestyle='--', label='Реальное значение')
        ax_ecc = axs[3].twinx()
        ax_ecc.plot(time_days, orbital['eccentricity'], 'r-', label='Эксцентриситет')
        axs[3].set_xlabel('Время, дни', fontsize=12)
        axs[3].set_ylabel('Полуось, м', fontsize=12)
        ax_ecc.set_ylabel('Эксцентриситет', color='r', fontsize=12)
        axs[3].legend(loc='upper left')
        ax_ecc.legend(loc='upper right')

        plt.suptitle('Динамика параметров БПП для системы Луна-Земля', fontsize=16)
        plt.tight_layout()
        plt.savefig('bpp_parameters.png', dpi=150)
        plt.close()

        # 5. Фазовый портрет
        radial_velocity = np.sum(moon_rel * velocities["Moon_relative"], axis=1) / earth_distance
        plt.figure(figsize=(10, 8))
        plt.scatter(earth_distance, radial_velocity, c=time_days, cmap='viridis', s=10)
        plt.colorbar(label='Время, дни')
        plt.axvline(x=3.844e8, color='r', linestyle='--', alpha=0.7)
        plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
        plt.xlabel('Расстояние Земля-Луна, м', fontsize=12)
        plt.ylabel('Радиальная скорость, м/с', fontsize=12)
        plt.title('Фазовый портрет движения Луны', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.savefig('phase_portrait.png', dpi=150)
        plt.close()

    def calculate_orbital_elements(self, trajectories, velocities):
        """Расчет орбитальных элементов Луны"""
        moon_rel = trajectories["Moon_relative"]
        moon_vel_rel = velocities["Moon_relative"]

        eccentricity = []
        semi_major_axis = []

        for i in range(len(moon_rel)):
            r_vec = moon_rel[i]
            r = np.linalg.norm(r_vec)
            v_vec = moon_vel_rel[i]
            v = np.linalg.norm(v_vec)

            # Гравитационный параметр Земли
            mu = self.G * self.system["Earth"]["mass"]

            # Угловой момент
            h_vec = np.cross(np.append(r_vec, 0), np.append(v_vec, 0))
            h = np.linalg.norm(h_vec)

            # Вектор эксцентриситета
            e_vec = ((v_vec.dot(v_vec) - mu/r) * r_vec - r_vec.dot(v_vec) * v_vec) / mu
            e = np.linalg.norm(e_vec)

            # Большая полуось
            p = h**2 / mu
            if e < 1:
                a = p / (1 - e**2)
            else:
                a = p / (e**2 - 1)

            eccentricity.append(e)
            semi_major_axis.append(a)

        return {
            "semi_major_axis": np.array(semi_major_axis),
            "eccentricity": np.array(eccentricity)
        }

    def run_full_analysis(self):
        """Полный цикл анализа: симуляция + визуализация"""
        # Запуск симуляции
        trajectories, velocities = self.run_simulation()

        # Визуализация орбиты с новыми параметрами
        self.plot_results(trajectories, velocities)  # Исправлено: добавлен velocities

        # Расчет орбитальных элементов
        orbital_elements = self.calculate_orbital_elements(trajectories, velocities)

        # Визуализация элементов орбиты
        if len(self.t_points) > 0:
            time_days = self.t_points / (24 * 3600)

            plt.figure(figsize=(14, 10))

            # Большая полуось
            plt.subplot(211)
            plt.plot(time_days, orbital_elements["semi_major_axis"])
            plt.axhline(y=3.844e8, color='r', linestyle='--', label='Наблюдаемое значение')
            plt.title('Большая полуось орбиты Луны', fontsize=14)
            plt.ylabel('a, м', fontsize=12)
            plt.grid(True)
            plt.legend()

            # Эксцентриситет
            plt.subplot(212)
            plt.plot(time_days, orbital_elements["eccentricity"])
            plt.axhline(y=0.0549, color='r', linestyle='--', label='Наблюдаемое значение')
            plt.title('Эксцентриситет орбиты Луны', fontsize=14)
            plt.ylabel('e', fontsize=12)
            plt.xlabel('Время, дни', fontsize=12)
            plt.grid(True)
            plt.legend()

            plt.tight_layout()
            plt.savefig('orbital_parameters.png', dpi=150)
            plt.close()
        else:
            print("Нет данных для построения графиков орбитальных параметров")

        return trajectories, velocities, orbital_elements

# Запуск симуляции
if __name__ == "__main__":
    print("Запуск иерархической модели БПП...")
    simulator = HierarchicalBPPSimulator()
    print("Проведение полного анализа...")
    trajectories, velocities, orbital_elements = simulator.run_full_analysis()
    print("Анализ завершен! Результаты сохранены в файлах:")
    print("- bpp_moon_orbit.png: Орбита Луны")
    print("- orbital_parameters.png: Параметры орбиты")
    print("- bpp_moon_animation.mp4: Анимация движения (если создана)")

Запуск иерархической модели БПП...
Проведение полного анализа...


Анализ завершен! Результаты сохранены в файлах:
- bpp_moon_orbit.png: Орбита Луны
- orbital_parameters.png: Параметры орбиты
- bpp_moon_animation.mp4: Анимация движения (если создана)
